In [2]:
import pandas as pd
import json
import collections
import csv

In [9]:
# get dialogue of each act
df = pd.read_csv("../data/dialoguebyact.csv")
df = df[df["act"] != 0.21] #ignore credits

# contributors of each act
contributors_dataset = pd.read_csv("../data/contributorForEachActFinal.csv")

# create gender file with contributors that never speak
gender_dataset = pd.read_csv("../data/gender.csv")
contributorWithoutDialogueGender = pd.read_csv("../data/contributorWithoutDialogueGender.csv")
gender_dataset2 = pd.concat([gender_dataset, contributorWithoutDialogueGender])

contributorsUppercase = contributors_dataset
# lowercase all the names so they can match
gender_dataset2["name"] = gender_dataset2["name"].str.lower()
contributors_dataset["name"] = contributors_dataset["name"].str.lower()

# merge contributor listing with gender listing
contributors_gender = contributors_dataset.merge(gender_dataset2, on="name")

In [28]:
df["name"] = df["name"].str.lower()
subjectsPerAct = df[(df["role"]=="subject")][["episode-act","name","gender","wordCount"]].groupby(["name","episode-act","gender"]).sum()["wordCount"].reset_index()

In [ ]:
contributors_gender.to_csv("../data/producersOfEachAct.csv", index=False)
subjectsPerAct.to_csv("../data/subjectsOfEachAct.csv", index=False)

In [ ]:
# getting reporter gender breakdown per act
producerGenders = []

for episode in range(1, 624):
    for act in range(0, 22):
        episodeAct = episode + float(act) * 0.01
        producers = contributors_gender[contributors_gender["episode-act"]== episodeAct]
        maleCount = 0
        femaleCount = 0
        for index, row in producers.iterrows():
#             if (row["name"] != "ira glass"):
            if row["gender"] == "M":
                maleCount += 1
            if row["gender"] == "F":
                femaleCount += 1
        total = float(femaleCount + maleCount)
        malepercent = -1
        femalepercent = -1
        if total != 0:
            malepercent = maleCount/total
            femalepercent = femaleCount/total
        producerGenders.append([episodeAct, malepercent, femalepercent, maleCount, femaleCount, total])

with open('../data/producersGender.csv', 'wb') as f: 
    w = csv.writer(f)
    w.writerow(["episode-act","male%","female%","male","female","total"])
    for list1 in producerGenders:
        w.writerow(list1)

In [ ]:
producerGenderData = pd.read_csv("../data/producersGender.csv")
producerGenderData = producerGenderData.drop_duplicates()

In [8]:
# exclude prologues, credits, and acts where no reporter is listed
producerGenderData[(producerGenderData["male%"] != -1)  & (producerGenderData["episode-act"] != 0.00) & (producerGenderData["episode-act"] != 0.21)].shape

(2467, 6)

In [9]:
# getting subject/interviewee gender breakdown per act
subjectsPerActAltered = []

for episode in range(1, 624):
    for act in range(0, 22):
        episodeAct = episode + float(act) * 0.01
        producers = contributors_gender[contributors_gender["episode-act"]== episodeAct]
        subjects = subjectsPerAct[subjectsPerAct["episode-act"] == episodeAct]
        for index, srow in subjects.iterrows():
            for index2, prow in producers.iterrows():
                if srow["name"].lower() != prow["name"].lower():
                    subjectsPerActAltered.append([srow["name"], episodeAct, srow["gender"], srow["wordCount"]])

with open('../data/subjectGendersWithoutProducers.csv', 'wb') as f: 
    w = csv.writer(f)
    w.writerow(["name", "episode-act","gender", "wordCount"])
    for list1 in subjectsPerActAltered:
        w.writerow(list1)

In [10]:
subjectsPerAct2 = pd.read_csv("../data/subjectGendersWithoutProducers.csv").groupby(["gender", "episode-act"])["wordCount"].sum().reset_index()

In [11]:
maleSubjects = subjectsPerAct2[subjectsPerAct2["gender"]=="M"]
femaleSubjects = subjectsPerAct2[subjectsPerAct2["gender"]=="F"]
mergedSubjects = maleSubjects.merge(femaleSubjects, on="episode-act", how="outer").fillna(0)
mergedSubjects["total"] = mergedSubjects["wordCount_x"]+mergedSubjects["wordCount_y"]
mergedSubjects["male%"] = mergedSubjects["wordCount_x"]/mergedSubjects["total"]
subjects = mergedSubjects[["episode-act","male%","wordCount_x","total"]]

In [12]:
# merge producer gender data and interviewee gender data into one spreadsheet
producerSubject = producerGenderData.merge(subjects, on="episode-act")[["episode-act", "male%_x","total_x","male%_y","wordCount_x","total_y"]]
producerSubject.columns = ["episode-act","percentMaleProducers","totalProducerCount","percentMaleSubjectDialogue","maleSubjectWordCount","totalSubjectWordCount"]
actInfo = pd.read_csv("../data/actInfo.csv")
mergedAct3 = producerSubject.merge(actInfo, on="episode-act")
mergedAct3 = mergedAct3[mergedAct3["act"] != 0]
mergedAct3["percentMaleProducers"] = mergedAct3["percentMaleProducers"]*100
mergedAct3["percentMaleSubjectDialogue"] = mergedAct3["percentMaleSubjectDialogue"]*100

In [13]:
# export spreadsheet for use
mergedAct3 = mergedAct3.sort_values("percentMaleSubjectDialogue", ascending = False)
mergedAct3.to_csv("../web/src/assets/data/act3.csv", index=False)

In [14]:
#average amount of male interviewee dialogue for male reported acts and female reported acts
male = mergedAct3[mergedAct3["percentMaleProducers"] == 100]
female = mergedAct3[(mergedAct3["percentMaleProducers"] == 0)]
male["percentMaleSubjectDialogue"].mean(), female["percentMaleSubjectDialogue"].mean()

(68.686856100131877, 64.744854353538258)

In [15]:
maleContributorCount = len(contributors_gender[contributors_gender["gender"]=="M"]["name"].drop_duplicates())
femaleContributorCount = len(contributors_gender[contributors_gender["gender"]=="F"]["name"].drop_duplicates())
#percent of contributors that are male
maleContributorCount/float(maleContributorCount+femaleContributorCount)

0.59472049689441

In [24]:
# % of contributors that are male per year
episodes = pd.read_table("../data/episodes.tsv")
contributorsByYear = contributors_gender.merge(episodes, on="episode")
contributorsByYear = contributorsByYear[["gender","name","year"]]

for year in range(1995,2018):
    subset = contributorsByYear[contributorsByYear["year"]== year]
    male = len(subset[subset["gender"]=="M"])
    female = len(subset[subset["gender"]=="F"])
    print year, male, female, male/float(female + male)


1995 941 23 0.976141078838
1996 192 189 0.503937007874
1997 79 34 0.699115044248
1998 119 32 0.788079470199
1999 62 48 0.563636363636
2000 33 78 0.297297297297
2001 84 21 0.8
2002 53 19 0.736111111111
2003 22 10 0.6875
2004 39 50 0.438202247191
2005 25 20 0.555555555556
2006 19 12 0.612903225806
2007 25 9 0.735294117647
2008 38 25 0.603174603175
2009 26 64 0.288888888889
2010 60 8 0.882352941176
2011 25 12 0.675675675676
2012 42 43 0.494117647059
2013 6 14 0.3
2014 17 43 0.283333333333
2015 24 15 0.615384615385
2016 16 13 0.551724137931
2017 3 5 0.375


In [34]:
# % of interviewees that are male per year
subjectsPerAct3 = df[(df["role"]=="subject")][["episode-act","episode","name","gender","wordCount"]].groupby(["name","episode-act","episode","gender"]).sum()["wordCount"].reset_index()
subjectsPerYear3 = subjectsPerAct3.merge(episodes, on="episode")
subjectsPerYear3 = subjectsPerYear3[["gender","name","year", "wordCount"]]
for year in range(1995,2018):
    subset = subjectsPerYear3[subjectsPerYear3["year"]== year]
    male = len(subset[subset["gender"]=="M"])
    female = len(subset[subset["gender"]=="F"])
    print str(year) + "\t" + str(male) + "\t" + str(female) + "\t" + str(male/float(female + male))

1995	42	20	0.677419354839
1996	258	189	0.577181208054
1997	309	174	0.639751552795
1998	198	157	0.557746478873
1999	222	163	0.576623376623
2000	246	114	0.683333333333
2001	258	164	0.611374407583
2002	288	145	0.665127020785
2003	265	131	0.669191919192
2004	225	144	0.609756097561
2005	183	97	0.653571428571
2006	107	39	0.732876712329
2007	204	122	0.625766871166
2008	236	120	0.662921348315
2009	344	150	0.696356275304
2010	349	137	0.718106995885
2011	335	223	0.600358422939
2012	264	123	0.682170542636
2013	411	209	0.662903225806
2014	335	184	0.645472061657
2015	423	245	0.633233532934
2016	303	165	0.647435897436
2017	193	104	0.649831649832


In [49]:
# % of subject dialogue that is male by year
df2 = df.merge(episodes, on="episode")
for year in range(1995, 2018):
    subjectsPerAct = df2[(df2["role"]=="subject") & (df2["year"]==year)][["episode-act","name","gender","wordCount"]].groupby(["name","episode-act","gender"]).sum()["wordCount"].reset_index()
    maleCount = subjectsPerAct[subjectsPerAct["gender"]=="M"]["wordCount"].sum()
    femaleCount = subjectsPerAct[subjectsPerAct["gender"]=="F"]["wordCount"].sum()
    print year, maleCount, femaleCount, maleCount/float(femaleCount + maleCount)

1995 13913 3089 0.818315492295
1996 62254 40030 0.608638692269
1997 84930 35801 0.703464727369
1998 74908 37208 0.668129437368
1999 68950 35678 0.659001414535
2000 56791 31029 0.64667501708
2001 56428 31242 0.643640926201
2002 63005 30129 0.67649837868
2003 52016 29682 0.636686332591
2004 66423 27046 0.710642031048
2005 76318 35809 0.680638918369
2006 36700 20103 0.646092635952
2007 69806 22102 0.759520389955
2008 62484 26791 0.699904788575
2009 71818 24061 0.749048279602
2010 68056 16774 0.80226335023
2011 66649 38175 0.635818133252
2012 66910 21774 0.75447656849
2013 74120 34574 0.681914365098
2014 67304 34009 0.66431751108
2015 83212 45389 0.647055621651
2016 73006 41015 0.640285561432
2017 48036 21227 0.693530456376


In [66]:
# % of contributor dialogue that is male
contributors1 = contributors_dataset.merge(df, on=["name","episode-act"], how="left").merge(episodes, on="episode")
contributors1
for year in range(1995, 2018):
    subjectsPerAct = contributors1[(contributors1["year"]==year)][["episode-act","name","gender","wordCount"]].groupby(["name","episode-act","gender"]).sum()["wordCount"].reset_index()
    maleCount = subjectsPerAct[subjectsPerAct["gender"]=="M"]["wordCount"].sum()
    femaleCount = subjectsPerAct[subjectsPerAct["gender"]=="F"]["wordCount"].sum()
    print year, maleCount, femaleCount, maleCount/float(femaleCount + maleCount)

1995 24771.0 8366.0 0.747532969189
1996 134982.0 40649.0 0.768554526251
1997 147067.0 48235.0 0.753023522545
1998 108558.0 51793.0 0.677002326147
1999 99406.0 40989.0 0.708045158303
2000 90528.0 42784.0 0.679068650984
2001 107117.0 40100.0 0.727612979479
2002 82744.0 50967.0 0.618827172035
2003 95330.0 40180.0 0.703490517305
2004 85916.0 39107.0 0.687201554914
2005 79854.0 39560.0 0.668715560989
2006 76434.0 19676.0 0.795276245968
2007 86443.0 32182.0 0.728708113804
2008 111262.0 31440.0 0.779680733276
2009 109232.0 41890.0 0.722806738926
2010 102701.0 37144.0 0.734391647896
2011 99156.0 58613.0 0.628488486331
2012 89383.0 33619.0 0.726679240988
2013 79518.0 58351.0 0.576764900014
2014 75391.0 52668.0 0.588720823995
2015 83178.0 67557.0 0.551816101105
2016 88665.0 74172.0 0.544501556771
2017 62563.0 31388.0 0.665910953582


In [63]:
# % of all dialogue that is male year by year
for year in range(1995, 2018):
    subjectsPerAct = df2[(df2["year"]==year) & (df2["name"]!="ira glass")][["episode-act","name","gender","wordCount"]].groupby(["name","episode-act","gender"]).sum()["wordCount"].reset_index()
    maleCount = subjectsPerAct[subjectsPerAct["gender"]=="M"]["wordCount"].sum()
    femaleCount = subjectsPerAct[subjectsPerAct["gender"]=="F"]["wordCount"].sum()
    print year, maleCount, femaleCount, maleCount/float(femaleCount + maleCount)

1995 29267 11455 0.718702421296
1996 156999 90264 0.634947404181
1997 190779 86941 0.686947285035
1998 134210 82430 0.619507016248
1999 131335 71962 0.646025273369
2000 113052 72512 0.609234549805
2001 131365 72431 0.6445906691
2002 107903 78710 0.578218023396
2003 141566 68848 0.672797437433
2004 113796 60318 0.653571797788
2005 128258 74281 0.633250880077
2006 88587 36659 0.707304025677
2007 135044 52392 0.720480590708
2008 130083 61623 0.678554661826
2009 124523 61780 0.668389666296
2010 146693 54004 0.730917751636
2011 131614 100597 0.566786241823
2012 144854 63210 0.696199246386
2013 145169 103971 0.582680420647
2014 138260 94453 0.594122373911
2015 149086 114723 0.56512855892
2016 127790 111172 0.534771218855
2017 88316 47932 0.648200340555


In [65]:
# % of all speakers that are male year by year
allspeakers = df[["episode-act","episode","name","gender","wordCount"]].groupby(["name","episode-act","episode","gender"]).sum()["wordCount"].reset_index()
allspeakers1 = allspeakers.merge(episodes, on="episode")
allspeakers1 = allspeakers1[["gender","name","year", "wordCount"]]
for year in range(1995,2018):
    subset = allspeakers1[allspeakers1["year"]== year]
    male = len(subset[subset["gender"]=="M"])
    female = len(subset[subset["gender"]=="F"])
    print str(year) + "\t" + str(male) + "\t" + str(female) + "\t" + str(male/float(female + male))

1995	91	26	0.777777777778
1996	504	231	0.685714285714
1997	564	208	0.730569948187
1998	378	179	0.678635547576
1999	401	194	0.673949579832
2000	391	147	0.726765799257
2001	438	192	0.695238095238
2002	428	170	0.715719063545
2003	450	164	0.732899022801
2004	357	178	0.667289719626
2005	323	121	0.727477477477
2006	216	52	0.805970149254
2007	347	147	0.702429149798
2008	383	147	0.722641509434
2009	497	186	0.727672035139
2010	528	167	0.759712230216
2011	513	275	0.651015228426
2012	438	160	0.732441471572
2013	596	278	0.681922196796
2014	501	230	0.6853625171
2015	590	303	0.660694288914
2016	455	217	0.677083333333
2017	277	119	0.699494949495
